In [ ]:

from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from datasets import load_from_disk

balanced_selfMA_Toxigen = load_from_disk("/content/drive/MyDrive/266_project/balanced_selfMA_ds_toxigen_quote")

In [ ]:
#Some basic EDA

print(balanced_selfMA_Toxigen)               # Dataset or DatasetDict
print(balanced_selfMA_Toxigen.column_names)  # columns

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3120
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 390
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 390
    })
})
{'train': ['text', 'label'], 'validation': ['text', 'label'], 'test': ['text', 'label']}


In [ ]:
balanced_selfMA_Toxigen['train'].to_csv("balanced_selfMA_Toxigen_train.csv")
balanced_selfMA_Toxigen['validation'].to_csv("balanced_selfMA_Toxigen_validation.csv")
balanced_selfMA_Toxigen['test'].to_csv("balanced_selfMA_Toxigen_test.csv")

Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

39488

In [ ]:
#convert entire dataset to CSV

import pandas as pd

dfs = []
for split in balanced_selfMA_Toxigen:
    df = balanced_selfMA_Toxigen[split].to_pandas()
    df["split"] = split
    dfs.append(df)

full_df = pd.concat(dfs, ignore_index=True)
full_df.to_csv("balanced_selfMA_Toxigen.csv", index=False)

In [ ]:
#read CSV
balanced_selfMA_Toxigen = pd.read_csv("/content/balanced_selfMA_Toxigen.csv")

In [ ]:
#sanity check
balanced_selfMA_Toxigen.head()

,text,label,split
0,"Wow, that's so complicated. I bet you really w...",1,train
1,I didn't know girls played video games.,1,train
2,The Women's studies department once held a San...,1,train
3,there is no difference in the quality of paren...,0,train
4,"Asians don't really speak proper English, do t...",2,train


In [ ]:
#see entire text column
for i, row in balanced_selfMA_Toxigen.iterrows():
    print(f"Label: {row['label']}")
    print(f"Text:  {row['text']}")
    print("-" * 80)

Label: 1
Text:  Wow, that's so complicated. I bet you really want to get married soon so you can have an easier one!
--------------------------------------------------------------------------------
Label: 1
Text:  I didn't know girls played video games.
--------------------------------------------------------------------------------
Label: 1
Text:  The Women's studies department once held a Sandwich-making day in his honor.
--------------------------------------------------------------------------------
Label: 0
Text:  there is no difference in the quality of parenting between gay and lesbian parents and heterosexual parents, and there is no difference in the quality of parenting between gay
--------------------------------------------------------------------------------
Label: 2
Text:  Asians don't really speak proper English, do they?
--------------------------------------------------------------------------------
Label: 1
Text:  Asian toddlers are the cutest things in the woorld. Ad

In [ ]:
#split datset according to label

import pandas as pd

# all label 0 and label 2
df_Toxigen = balanced_selfMA_Toxigen[balanced_selfMA_Toxigen['label'].isin([0, 2])]

# Dataset B = all label 1
df_selfMA = balanced_selfMA_Toxigen[balanced_selfMA_Toxigen['label'] == 1]

print("df_Toxigen (labels 0 and 2):", df_Toxigen.shape)
print("Dataset B (label 1):", df_selfMA.shape)


df_Toxigen (labels 0 and 2): (2600, 3)
Dataset B (label 1): (1300, 3)


In [ ]:
#installs for readability tests

!pip install -q textstat tqdm

import numpy as np
from tqdm import tqdm
import textstat
import math

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.4 MB/s eta 0:00:00


In [ ]:
#reset index

df_Toxigen = df_Toxigen.reset_index(drop=True)
df_selfMA  = df_selfMA.reset_index(drop=True)

print(f"Rows in Toxigen (0/2): {len(df_Toxigen)}")
print(f"Rows in SelfMA (1):    {len(df_selfMA)}")

Rows in Toxigen (0/2): 2600
Rows in SelfMA (1):    1300


In [ ]:
#readability tests

def safe_metric(func, text):
    """Run a textstat metric and return NaN if it errors."""
    try:
        return func(text)
    except Exception:
        return np.nan

def readability_metrics(text: str):
    """
    Compute readability metrics for ANY text (even short fragments).
    Returns a dict; errors in individual metrics become NaN.
    """
    text = str(text).strip()

    return {
        "flesch_reading_ease":         safe_metric(textstat.flesch_reading_ease, text),
        "flesch_kincaid_grade":        safe_metric(textstat.flesch_kincaid_grade, text),
        "gunning_fog":                 safe_metric(textstat.gunning_fog, text),
        "smog_index":                  safe_metric(textstat.smog_index, text),
        "coleman_liau_index":          safe_metric(textstat.coleman_liau_index, text),
        "automated_readability_index": safe_metric(textstat.automated_readability_index, text),
        "dale_chall_readability":      safe_metric(textstat.dale_chall_readability_score, text),
        "linsear_write":               safe_metric(textstat.linsear_write_formula, text),
        "difficult_words":             safe_metric(textstat.difficult_words, text),
        "avg_sentence_length":         safe_metric(textstat.avg_sentence_length, text),
        "avg_syllables_per_word":      safe_metric(textstat.avg_syllables_per_word, text),
        "text_standard_grade":         safe_metric(lambda t: textstat.text_standard(t, float_output=False), text),
        "word_count":                  len(text.split()) if text else 0,
        "char_count":                  len(text),
    }


def compute_readability_df(df_in: pd.DataFrame, text_col="text") -> pd.DataFrame:
    rows = []
    for t in tqdm(df_in[text_col].tolist(), desc="Computing readability"):
        rows.append(readability_metrics(t))
    return pd.DataFrame(rows)


# --- 3) Compute metrics for your two subsets ---
read_Toxigen = compute_readability_df(df_Toxigen, text_col="text")   # labels 0 + 2
read_selfMA  = compute_readability_df(df_selfMA,  text_col="text")   # label 1


# Attach metrics back to original rows
df_Toxigen_metrics = pd.concat([df_Toxigen.reset_index(drop=True), read_Toxigen], axis=1)
df_selfMA_metrics  = pd.concat([df_selfMA.reset_index(drop=True),  read_selfMA],  axis=1)


# --- 4) Compare groups: means, std, Cohen's d ---

NUMERIC_COLS = [
    "flesch_reading_ease",
    "flesch_kincaid_grade",
    "gunning_fog",
    "smog_index",
    "coleman_liau_index",
    "automated_readability_index",
    "dale_chall_readability",
    "linsear_write",
    "difficult_words",
    "avg_sentence_length",
    "avg_syllables_per_word",
    "word_count",
    "char_count",
]


summary = pd.DataFrame({
    "metric": NUMERIC_COLS,
    "Toxigen_mean": [df_Toxigen_metrics[c].mean() for c in NUMERIC_COLS],
    "SelfMA_mean":  [df_selfMA_metrics[c].mean() for c in NUMERIC_COLS],
})


print("\n=== Readability Comparison (Toxigen 0/2 vs SelfMA 1) ===")
print(summary)


# save outputs
df_Toxigen_metrics.to_csv("readability_Toxigen_0_2.csv", index=False)
df_selfMA_metrics.to_csv("readability_selfMA_1.csv", index=False)
summary.to_csv("readability_comparison_summary.csv", index=False)

print("\nSaved files:")
print(" - readability_Toxigen_0_2.csv")
print(" - readability_selfMA_1.csv")


Computing readability:   0%|          | 0/2600 [00:00<?, ?it/s]/tmp/ipython-input-2313493660.py:6: DeprecationWarning: The 'avg_sentence_length' method has been deprecated due to being the same as 'words_per_sentence'. This method will be removed in thefuture.
  return func(text)
Computing readability: 100%|██████████| 1300/1300 [00:00<00:00, 6365.41it/s]


=== Readability Comparison (Toxigen 0/2 vs SelfMA 1) ===
                         metric  Toxigen_mean  SelfMA_mean
0           flesch_reading_ease     67.059879    85.381980
1          flesch_kincaid_grade      8.043703     3.751233
2                   gunning_fog     10.225746     6.065988
3                    smog_index      9.121564     6.312119
4            coleman_liau_index      6.878224     4.094519
5   automated_readability_index      7.738546     4.284074
6        dale_chall_readability      7.730784     6.465403
7                 linsear_write      9.484135     4.537381
8               difficult_words      2.764615     1.852308
9           avg_sentence_length     16.758910     9.664685
10       avg_syllables_per_word      1.446419     1.319662
11                   word_count     18.087692    15.467692
12                   char_count     94.568462    82.409231

Saved files:
 - readability_Toxigen_0_2.csv
 - readability_selfMA_1.csv
